# Extracting relevant NER

In [10]:
import datetime

import pandas as pd
import datasets
from functools import partial

from backend.keyword.extraction import KeywordExtractor

### Theory behind

Представляется, что мы можем судить о важности слова исходя из сравнения распределения его частотности по документам.
Проще говоря, если токен является ключевым для какого-то подкорпуса, то его встречаемость в этом подкорпусе будет
сильно отличаться от его обычной встречаемости. И наоборот - если слово не является каким-то детерминирующим, то его 
встречаемость в подкорпусе будет примерно такой же, как и везде.

$H_0:$ частотность слова $t$ в подкорусе $d$ статистически значимо превышает его частотность в документах из коллекции $D$

$H_1:$ частотность слова $t$ в подкорусе $d$ не превышает его частотность в документах из коллекции $D$
	
Нас интересуют следующие значения:

|    | d | D |
|----| ------| --- |
| t  | j | k |
| ~t | l | m |

* $j$ - количество вхождений слова $t$ в подкорус $d$ 
* $k$ - количество вхождений слова $t$ во все тексты коллекции $D$
* $l$ - количество вхождений всех слов, кроме $t$, в подкорус $d$ 
* $m$ - количество вхождений всех слов, кроме $t$, в коллекцию документов $D$

**Ожидаемая** частотность слова $t$ в документе $d$ рассчитывается следующим образом: 

$$Expected = \frac{( j + k ) \times (j + l)}{j + k + l + m}$$

Для того, чтобы понять, *достаточно ли сильно* ожидаемая частотность превышает наблюдаемую, необходимо воспользоваться статистическим критерием.
Для сравнения распределений хорошо подходит $\chi^2$ (хи-квадрат). Посчитать его можно по следующей сокращенной формуле:

$$\chi^2 = \frac{(observed - expected)^2}{expected}$$

При этом за наблюдаемую (observed) частотность мы принимаем то, сколько раз слово встретилось в анализируемом подкорпусе.

Статистически значимыми словами считаются те, для которых значение $\chi^2$ превышает некое критическое значение. 

При этом выбор критического значения зависит от того, насколько мы готовы принять риск не распознать действительно важные слова как таковые. 
Проще говоря, чем выше критическое значение, тем с большей вероятностью мы упустим некоторые из ключевых слов. Но брать слишком 
низкие критические значения тоже чревато: в этом случае растет риск назвать ключевыми те слова, которые ими не являются.  

Вероятность назвать неключевое слово ключевым называется уровнем значимости.
Ниже представлен словарь с критическими значениями для разных уровней значимости: 
`CRITERION = {0.05: 3.842, 0.01: 6.635, 0.001: 10.828}`
Здесь ключи - это уровень значимости, а значение - критическая точка хи-квадрат.

Ссылка на оригинальную статью: https://cyberleninka.ru/article/n/raspredelenie-hi-kvadrat-i-vzveshivanie-terminov

### Loading Data

In [3]:
CACHE_DIR = 'data'

all_datasets = datasets.load_dataset('tner/tweetner7', cache_dir=CACHE_DIR)
train_dataset = all_datasets['train_all']

label_2_id = {
                 "B-corporation": 0,
                 "B-creative_work": 1,
                 "B-event": 2,
                 "B-group": 3,
                 "B-location": 4,
                 "B-person": 5,
                 "B-product": 6,
                 "I-corporation": 7,
                 "I-creative_work": 8,
                 "I-event": 9,
                 "I-group": 10,
                 "I-location": 11,
                 "I-person": 12,
                 "I-product": 13,
                 "O": 14
             }
id_2_label = {v: k for k, v in label_2_id.items()}

def to_bio(row, labels_mapping: dict):
    bio_labels = []
    for label in row['tags']:
        bio_labels.append(labels_mapping[label])
    row['bio_labels'] = bio_labels
    return row

converter_fn = partial(to_bio, labels_mapping={v: k for k, v in label_2_id.items()})
train_df = pd.DataFrame(train_dataset.map(converter_fn))[['tokens', 'date', 'bio_labels']]
train_df["date"] = pd.to_datetime(train_df.date)

In [4]:
train_df.head(5)

,tokens,date,bio_labels
0,"[Morning, 5km, run, with, {{USERNAME}}, for, b...",2019-10-13,"[O, O, O, O, O, O, O, O, O, O, B-event, O, B-e..."
1,"[President, Trump, Arrives, at, UFC, 244, in, ...",2019-11-03,"[B-person, I-person, O, O, B-corporation, I-co..."
2,"["", I, 've, been, in, law, enforcement, for, 2...",2020-05-31,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[I, got, mine, yesterday, !, ****, Doctors, sa...",2019-10-06,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Mayo, Breast, Cancer, Vaccine, Could, Be, Ava...",2019-10-13,"[B-product, I-product, I-product, I-product, O..."


### Extracting keywords

Let's define period of interest

In [11]:
start, end = '2019-10-10', '2019-10-20'
x = datetime.datetime(*list(map(int, start.split('-'))))
y = datetime.datetime(*list(map(int, end.split('-'))))

Now we can extract relevant NER along with their category and relevance score:

In [13]:
extractor = KeywordExtractor(data=train_df)
relevant_ner = extractor.extract_relevant(start=x, end=y)

In [14]:
pd.DataFrame(relevant_ner)

,entity,score,category
0,Breaking Bad,53.828056,creative_work
1,{@YouTube@},52.824189,product
2,El Camino,22.664445,creative_work
3,{@Aaron Paul@},14.165278,person
4,ElCamino,11.332222,creative_work
5,{@Breaking Bad@},11.332222,creative_work
6,Seahawks,10.786720,group
7,{@Poshmark@},9.567815,product
8,BreakingBad,8.499167,creative_work
9,bra,8.499167,product
